In [ ]:
import http.client
import mimetypes
import requests
import json
import csv
import pandas as pd
from datetime import date
from datetime import timedelta
from datetime import datetime
import time
import telegram
from urllib.parse import urlencode
import multiprocessing
import threading
bot = telegram.Bot(token = '1904162812:AAFJUKftPgt7KLs4aLHBpWV6_qkH5l4WRug')

In [ ]:
new_text = "BHEL gave returns today"

In [ ]:
bot.sendMessage(chat_id = '-486459146', text=new_text)

In [ ]:
def gettoken():
    conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
    url = "apiconnect.angelbroking.com/rest/auth/angelbroking/user/v1/loginByPassword"
    payload = {"clientcode":"A594088",
                "password":"Angel.jav123"}
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'X-UserType': 'USER',
        'X-SourceID': 'WEB',
        'X-ClientLocalIP': '192.168.1.11',
        'X-ClientPublicIP': '49.205.84.156',
        'X-MACAddress': '38:f9:d3:49:cd:8e',
        'X-PrivateKey': 'GyFrQxvG'
      }

    conn.request("POST", "/rest/auth/angelbroking/user/v1/loginByPassword",json.dumps(payload),headers)

    res = conn.getresponse()
    data = res.read()
    print(data)
    df = pd.read_json(data)
    token = 'Bearer ' + df.loc['jwtToken', 'data']
    return token

In [ ]:
def getdata(token, symtoken, fromdate, todate, freq):
    conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
    url = "apiconnect.angelbroking.com/rest/auth/angelbroking/user/v1/loginByPassword"
    payload = {
         "exchange": "NSE",
         "symboltoken": symtoken,
         "interval": freq,
         "fromdate": fromdate, #format: "2021-02-10 09:15"
         "todate": todate, # format: "2021-03-10 15:30"
    }

    headers = {
      'X-PrivateKey': 'GyFrQxvG',
      'Accept': 'application/json',
      'X-SourceID': 'WEB',
      'Authorization': token,
      'X-ClientLocalIP': '192.168.0.101',
      'X-ClientPublicIP': '114.143.211.242',
      'X-MACAddress': 'a4:83:e7:63:79:73',
      'X-UserType': 'USER',
      'Accept': 'application/json',
      'X-SourceID': 'WEB',
      'Content-Type': 'application/json'
    }

    conn.request("POST", "/rest/secure/angelbroking/historical/v1/getCandleData", json.dumps(payload), headers)    
    res = conn.getresponse()
    data = res.read()
    #print(data)
    return data

In [ ]:
def getsymtoken(symbol):
    ins = pd.read_json('/Users/abhinahar/Downloads/tradeable_instruments.json')
    symtoken = ins.loc[ins[ins['symbol'] == symbol+'-EQ'].index, 'token'].item()
    return symtoken

In [ ]:
def main(results, tradedate):
    start = time.time()
    token  = gettoken()
    global final
    for index, rows in results.iterrows():
        #try:
        columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        symtoken = getsymtoken(results.loc[index, 'symbol'])
        fromdate = str(tradedate) + ' 09:15'
        todate = str(tradedate) + ' 10:00'
        results.loc[index, 'date'] = tradedate
        data = getdata(token, symtoken, fromdate, todate, 'ONE_MINUTE')
        tmp = pd.read_json(data)
        a = list(tmp['data'])
        df = pd.DataFrame(a, columns = columns)
        df.insert(1, 'time', '')
        df['time'] = df['date'].astype(str).str[11:16]
        df['date'] = df['date'].astype(str).str[0:10]
        twomin = pd.DataFrame(columns=df.columns)
        for i, j in zip(range(0, len(df), 2), range(0, 30, 1)):
            twomin.loc[j, 'date'] = df.loc[i, 'date']
            twomin.loc[j, 'time'] = df.loc[i, 'time']
            twomin.loc[j, 'high'] = max(df.loc[i, 'high'], df.loc[i+1, 'high'])
            twomin.loc[j, 'low'] = min(df.loc[i, 'low'], df.loc[i+1, 'low'])
            twomin.loc[j, 'open'] = df.loc[i, 'open']
            twomin.loc[j, 'close'] = df.loc[i+1, 'close']
            twomin.loc[j, 'volume'] = df.loc[i, 'volume'] + df.loc[i+1, 'volume']
        backtest = twomin.copy()
        backtest.insert(len(backtest.columns), 'buy', '')
        backtest.insert(len(backtest.columns), 'sell', '')
        buypos = False
        sellpos = False
        for ind, rows in backtest.iterrows():
            exitprice = 0
            entryprice = 0
            exittime = 0
            if ind == 0:
                continue
            if buypos == False and sellpos == False:
                if backtest.loc[ind, 'high'] >= backtest.loc[ind-1, 'high']:
                    backtest.loc[ind, 'buy'] = 'OPEN'
                    entrytime = 0
                    posopen = buyopen = backtest.loc[ind-1, 'high']
                    buytarget = buyopen * 1.005
                    buystoploss = 0.995 * buyopen
                    entrytime = backtest.loc[ind, 'time']
                    buypos = True
                    position = 'long'
                    continue
                elif backtest.loc[ind, 'low'] <= backtest.loc[ind-1, 'low']:
                    backtest.loc[ind, 'sell'] = 'OPEN'
                    entrytime = 0
                    posopen = sellopen = backtest.loc[ind-1, 'low']
                    selltarget = sellopen * 0.995
                    sellstoploss = 1.005 * sellopen
                    entrytime = backtest.loc[ind, 'time']
                    sellpos = True
                    position = 'short'
                    continue
            if buypos == True:
                if backtest.loc[ind, 'high'] >= buytarget:
                    backtest.loc[ind, 'buy'] = backtest.loc[ind, 'buy'] + ' +CLOSE'
                    exittime = backtest.loc[ind, 'time']
                    exitprice = buytarget
                    success = 'YES'
                    percent = (buytarget-buyopen)/buyopen
                    break
                elif backtest.loc[ind, 'low'] <= buystoploss:
                    backtest.loc[ind, 'buy'] = backtest.loc[ind, 'buy'] + ' -CLOSE'
                    exittime = backtest.loc[ind, 'time']
                    exitprice = buystoploss
                    success = 'NO'
                    percent = (buystoploss-buyopen)/buyopen
                    break
            if sellpos == True:
                if backtest.loc[ind, 'high'] >= sellstoploss:
                    backtest.loc[ind, 'sell'] = backtest.loc[ind, 'sell'] + ' -CLOSE'
                    exittime = backtest.loc[ind, 'time']
                    exitprice = sellstoploss
                    success = 'NO'
                    percent = (sellopen-sellstoploss)/sellopen
                    break
                elif backtest.loc[ind, 'low'] <= selltarget:
                    backtest.loc[ind, 'sell'] = backtest.loc[ind, 'sell'] + ' +CLOSE'
                    exittime = backtest.loc[ind, 'time']
                    exitprice = selltarget
                    success = 'YES'
                    percent = (sellopen-selltarget)/sellopen
                    break
            if ind == 22:
                exitprice = backtest.loc[ind, 'close']
                success = 'NO'
                exittime = backtest.loc[ind, 'time']
                if sellpos == True:
                    percent = (sellopen-backtest.loc[ind, 'close'])/sellopen
                elif buypos == True:
                    percent = (backtest.loc[ind, 'close']-buyopen)/buyopen
                else:
                    percent = 0

        results.loc[index, 'entry time'] = entrytime
        results.loc[index, 'exit time'] = exittime
        results.loc[index, 'position'] = position
        results.loc[index, 'entry price'] = posopen
        results.loc[index, 'exit price'] = exitprice
        results.loc[index, 'success'] = success
        results.loc[index, 'percentage'] = percent
        # except:
        #     print('NOT PERFORMED')
        #     results.loc[index, 'success'] = 'NA'
        #     continue
    return results

In [ ]:
def screener(ticky,token,tradedate):
    comp = pd.read_csv('/Users/abhinahar/Downloads/f&o companies{}.csv'.format(ticky), index_col = 0)
    print("Abhishek")
    global final
    scan = False
    cols = ['symbol', 'date', 'onemin avg vol', '1min trading day vol', 'candle color', 'open', 'close', 'percentage first candle', 'vol ratio', 'entry time', 'exit time', 'position', 'entry price', 'exit price', 'success', 'percentage']
    columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    results = pd.DataFrame(columns=cols)
    for index, rows in comp.iterrows():
        #try:
        print(comp.loc[index, 'Symbol'])
        columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        symtoken = getsymtoken(comp.loc[index, 'Symbol'])
        fromdate = str(tradedate-timedelta(days = 28)) + ' 09:15'
        todate = str(tradedate-timedelta(days = 1)) + ' 15:30'
        data = getdata(token, symtoken, fromdate, todate, 'ONE_MINUTE',)
        tmp = pd.read_json(data)
        a = list(tmp['data'])
        df2 = pd.DataFrame(a, columns = columns)
        avgvoli = df2['volume'].mean()
        results.loc[index, 'onemin avg vol'] = avgvoli
        results.loc[index, 'symbol'] = comp.loc[index, 'Symbol']
        results.loc[index, 'date'] = tradedate
        #except:
            #print('NOT PERFORMED')
            #continue
    cols = ['symbol', 'date','onemin avg vol', '1min trading day vol', 'candle color', 'open', 'close', 'percentage first candle', 'vol ratio']
    fromdate = str(tradedate) + ' 09:15'
    todate = str(tradedate) + ' 09:15'
    time.sleep(0.5)
    while(scan == False):
        start = time.time()
        if datetime.now().minute >= 16:
            for index, rows in comp.iterrows():
                try:
                    symtoken = getsymtoken(comp.loc[index,'Symbol'])
                    data = getdata(token, symtoken, fromdate, todate, 'ONE_MINUTE')
                    print(comp.loc[index, 'Symbol'] + " SCAN")
                    tmp = pd.read_json(data)
                    a = list(tmp['data'])
                    df = pd.DataFrame(a, columns = columns)
                    results.loc[index, '1min trading day vol'] = df.loc[0, 'volume']
                    results.loc[index,'close'] = df.loc[0, 'close']
                    results.loc[index,'open'] = df.loc[0, 'open']
                    if df.loc[0, 'open'] - df.loc[0, 'close'] < 0:
                        results.loc[index, 'candle color'] = 'G'
                        results.loc[index, 'percentage first candle'] = round((df.loc[0, 'close']/df.loc[0, 'open']-1)*100,2)
                    else:
                        results.loc[index, 'candle color'] = 'R'
                        results.loc[index, 'percentage first candle'] = round((df.loc[0, 'open']/df.loc[0, 'close']-1)*100,2)
                    results.loc[index, 'vol ratio'] = results.loc[index, '1min trading day vol']/results.loc[index, 'onemin avg vol']          
                    time.sleep(0.25) 
                except:
                    print("--------------------NOT PERFORMED--------------------")
                    data = getdata(token, symtoken, fromdate, todate, 'ONE_MINUTE')
                    print(comp.loc[index, 'Symbol'] + " SCAN")
                    tmp = pd.read_json(data)
                    a = list(tmp['data'])
                    df = pd.DataFrame(a, columns = columns)
                    results.loc[index, '1min trading day vol'] = df.loc[0, 'volume']
                    results.loc[index,'close'] = df.loc[0, 'close']
                    results.loc[index,'open'] = df.loc[0, 'open']
                    if df.loc[0, 'open'] - df.loc[0, 'close'] < 0:
                        results.loc[index, 'candle color'] = 'G'
                        results.loc[index, 'percentage first candle'] = round((df.loc[0, 'close']/df.loc[0, 'open']-1)*100,2)
                    else:
                        results.loc[index, 'candle color'] = 'R'
                        results.loc[index, 'percentage first candle'] = round((df.loc[0, 'open']/df.loc[0, 'close']-1)*100,2)
                    results.loc[index, 'vol ratio'] = results.loc[index, '1min trading day vol']/results.loc[index, 'onemin avg vol']          
                    time.sleep(0.25)
            scan = True
    print('Time taken: ', round(time.time()-start,2), 's')
    results.to_csv('/Users/abhinahar/Desktop/EXtra files/MultiProcess_{}_{}.csv'.format(tradedate,ticky))
    

In [ ]:
start = time.time()
token = gettoken()
tradedate = date(2021,11,8)
cols = ['symbol', 'date', 'onemin avg vol', '1min trading day vol', 'candle color', 'open', 'close', 'percentage first candle', 'vol ratio', 'entry time', 'exit time', 'position', 'entry price', 'exit price', 'success', 'percentage']
final = pd.DataFrame(columns=cols)
# for i in range(20):
#     if ((tradedate == date(2021,8,16) or (tradedate == date(2021,8,19)) or (tradedate == date(2021,9,10)) or (tradedate.weekday() > 4))):
#         tradedate = tradedate + timedelta(days=1)
#         continue
    
print(tradedate)
p1 = multiprocessing.Process(target=screener, args=(1, token, tradedate))
p2 = multiprocessing.Process(target=screener, args=(2, token, tradedate))
p3 = multiprocessing.Process(target=screener, args=(3, token, tradedate))

p1.start()

p2.start()

p3.start()

p1.join()
p2.join()
p3.join()

df1 = pd.read_csv('/Users/abhinahar/Desktop/EXtra files/MultiProcess_{}_1.csv'.format(tradedate))
df2 = pd.read_csv('/Users/abhinahar/Desktop/EXtra files/MultiProcess_{}_2.csv'.format(tradedate))
df3 = pd.read_csv('/Users/abhinahar/Desktop/EXtra files/MultiProcess_{}_3.csv'.format(tradedate))

df2 = df2.append(df3,ignore_index=1)
df1 = df1.append(df2,ignore_index=1)

df1 = df1[ df1['percentage first candle'] > 0.75 ]
df1 = df1.sort_values(by='vol ratio', ascending=False)
df1.to_csv('/Users/abhinahar/Downloads/Multiprocesses/MultiProcess_{}_final.csv'.format(tradedate))
df1 = df1.iloc[:2]
final = final.append(main(df1, tradedate), ignore_index=1)
#tradedate = tradedate + timedelta(days=1)
final.to_csv('/Users/abhinahar/Downloads/MultiProcess_Backtest{}_final.csv'.format(datetime.now()))
print("\n\nTotal Program Runtime = " + str(round(time.time()-start)) + "secs")
    


In [ ]:
final.to_csv('/Users/abhinahar/Downloads/MultiProcess_Backtest{}_final.csv'.format(datetime.now()))

In [ ]:
print(gettoken())

In [ ]:
print(final)
final.to_csv('/Users/abhinahar/Downloads/MultiProcess_Backtest{}_final.csv'.format(datetime.now()))

In [ ]:
def volume_ratio():
    start = time.time()
    token  = gettoken()
    comp = pd.read_csv('/Users/abhinahar/Downloads/backtest_filterdates.csv')
    comp = comp.drop_duplicates()
    cols = ['symbol', 'date', 'onemin avg vol', 'trading day vol', 'candle color', 'open', 'close', 'percentage first candle']
    results = pd.DataFrame(columns=cols)
    for index, rows in comp.iterrows():
        try:
            print(comp.loc[index, 'Symbol'])
            columns = ['date', 'open', 'high', 'low', 'close', 'volume']
            symtoken = getsymtoken(comp.loc[index, 'Symbol'])
            fromdate = comp.loc[index, 'monthlyfrom'] + ' 09:15'
            todate = comp.loc[index, 'monthlyto'] + ' 15:30'
            data = getdata(token, symtoken, fromdate, todate, 'ONE_MINUTE',)
            tmp = pd.read_json(data)
            a = list(tmp['data'])
            df = pd.DataFrame(a, columns = columns)
            avgvol = df['volume'].mean()
            results.loc[index, 'onemin avg vol'] = avgvol
            fromdate = comp.loc[index, 'trading day'] + ' 09:15'
            todate = comp.loc[index, 'trading day'] + ' 09:30'
            data = getdata(token, symtoken, fromdate, todate, 'ONE_MINUTE')
            tmp = pd.read_json(data)
            a = list(tmp['data'])
            df = pd.DataFrame(a, columns = columns)
            results.loc[index, 'trading day vol'] = df.loc[0, 'volume']
            if df.loc[0, 'open'] - df.loc[0, 'close'] < 0:
                results.loc[index, 'candle color'] = 'G'
                results.loc[index, 'percentage first candle'] = df.loc[0, 'close']/df.loc[0, 'open']
                results.loc[index,'close'] = df.loc[0, 'close']
                results.loc[index,'open'] = df.loc[0, 'open']
            else:
                results.loc[index, 'candle color'] = 'R'
                results.loc[index, 'percentage first candle'] = df.loc[0, 'open']/df.loc[0, 'close']
                results.loc[index,'close'] = df.loc[0, 'close']
                results.loc[index,'open'] = df.loc[0, 'open']
            df.insert(1, 'time', '')
            df['time'] = df['date'].astype(str).str[11:16]
            df['date'] = df['date'].astype(str).str[0:10]
            # twomin = pd.DataFrame(columns=df.columns)
            # twomin = pd.DataFrame(columns=df.columns)
            # for i, j in zip(range(0, len(df), 2), range(0, 13, 1)):
            #     twomin.loc[j, 'date'] = df.loc[i, 'date']
            #     twomin.loc[j, 'time'] = df.loc[i, 'time']
            #     twomin.loc[j, 'high'] = max(df.loc[i, 'high'], df.loc[i+1, 'high'])
            #     twomin.loc[j, 'low'] = min(df.loc[i, 'low'], df.loc[i+1, 'low'])
            #     twomin.loc[j, 'open'] = df.loc[i, 'open']
            #     twomin.loc[j, 'close'] = df.loc[i+1, 'close']
            #     twomin.loc[j, 'volume'] = df.loc[i, 'volume'] + df.loc[i+1, 'volume']
            # backtest = twomin.copy()
            # backtest.insert(len(backtest.columns), 'buy', '')
            # backtest.insert(len(backtest.columns), 'sell', '')
            # buypos = False
            # sellpos = False
            # for ind, rows in backtest.iterrows():
            #     if ind == 0:
            #         results.loc[index, 'trading day vol'] = backtest.loc[ind, 'volume']
            #         if results.loc[index, 'trading day vol'] >= 0.2 * results.loc[index, 'daily avg vol'] and results.loc[index, 'daily avg vol'] >= 250000:
            #             results.loc[index, 'volfilter'] = 'YES'
            #         else:
            #             results.loc[index, 'volfilter'] = 'NO'
            #     exitprice = 0
            #     entryprice = 0
            #     exittime = 0
            #     if ind == 0:
            #         continue
            #     if buypos == False and sellpos == False:
            #         if backtest.loc[ind, 'high'] >= backtest.loc[ind-1, 'high']:
            #             backtest.loc[ind, 'buy'] = 'OPEN'
            #             entrytime = 0
            #             posopen = buyopen = backtest.loc[ind-1, 'high']
            #             buytarget = buyopen * 1.01
            #             buystoploss = 0.99 * buyopen
            #             entrytime = backtest.loc[ind, 'time']
            #             buypos = True
            #             position = 'long'
            #         elif backtest.loc[ind, 'low'] <= backtest.loc[ind-1, 'low']:
            #             backtest.loc[ind, 'sell'] = 'OPEN'
            #             entrytime = 0
            #             posopen = sellopen = backtest.loc[ind-1, 'low']
            #             selltarget = sellopen * 0.99
            #             sellstoploss = 1.05 * sellopen
            #             entrytime = backtest.loc[ind, 'time']
            #             sellpos = True
            #             position = 'short'
            #     if buypos == True:
            #         if backtest.loc[ind, 'high'] >= buytarget:
            #             backtest.loc[ind, 'buy'] = backtest.loc[ind, 'buy'] + ' +CLOSE'
            #             exittime = backtest.loc[ind, 'time']
            #             exitprice = buytarget
            #             success = 'YES'
            #             percent = (buytarget-buyopen)/buyopen
            #             break
            #         elif backtest.loc[ind, 'low'] <= buystoploss:
            #             backtest.loc[ind, 'buy'] = backtest.loc[ind, 'buy'] + ' -CLOSE'
            #             exittime = backtest.loc[ind, 'time']
            #             exitprice = buystoploss
            #             success = 'NO'
            #             percent = (buystoploss-buyopen)/buyopen
            #             break
            #     if sellpos == True:
            #         if backtest.loc[ind, 'high'] >= sellstoploss:
            #             backtest.loc[ind, 'sell'] = backtest.loc[ind, 'sell'] + ' -CLOSE'
            #             exittime = backtest.loc[ind, 'time']
            #             exitprice = sellstoploss
            #             success = 'NO'
            #             percent = (sellopen-sellstoploss)/sellopen
            #             break
            #         elif backtest.loc[ind, 'low'] <= selltarget:
            #             backtest.loc[ind, 'sell'] = backtest.loc[ind, 'sell'] + ' +CLOSE'
            #             exittime = backtest.loc[ind, 'time']
            #             exitprice = selltarget
            #             success = 'YES'
            #             percent = (sellopen-selltarget)/sellopen
            #             break
            #     if ind == 7:
            #         exitprice = backtest.loc[ind, 'close']
            #         success = 'NO'
            #         exittime = backtest.loc[ind, 'time']
            #         if sellpos == True:
            #             percent = (sellopen-backtest.loc[ind, 'close'])/sellopen
            #         elif buypos == True:
            #             percent = (backtest.loc[ind, 'close']-buyopen)/buyopen
            #         else:
            #             percent = 0

            results.loc[index, 'symbol'] = comp.loc[index, 'Symbol']
            results.loc[index, 'date'] = comp.loc[index, 'trading day']
        except:
            print('NOT PERFORMED')
            continue
    print('Time taken: ', time.time()-start)
    return results

In [ ]:
results = volume_ratio()
print(results)

In [ ]:
results.to_csv('/Users/abhinahar/Downloads/Combine.csv')

In [ ]:
tmp = pd.read_json(data)
a = list(tmp['data'])
columns = ['date', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(a, columns = columns)
df.insert(1, 'time', '')
df['time'] = df['date'].astype(str).str[11:16]
df['date'] = df['date'].astype(str).str[0:10]
twomin = pd.DataFrame(columns=df.columns)

In [ ]:
for i, j in zip(range(0, len(df), 2), range(0, 13, 1)):
    twomin.loc[j, 'date'] = df.loc[i, 'date']
    twomin.loc[j, 'time'] = df.loc[i, 'time']
    twomin.loc[j, 'high'] = max(df.loc[i, 'high'], df.loc[i+1, 'high'])
    twomin.loc[j, 'low'] = min(df.loc[i, 'low'], df.loc[i+1, 'low'])
    twomin.loc[j, 'open'] = df.loc[i, 'open']
    twomin.loc[j, 'close'] = df.loc[i+1, 'close']
twomin = twomin.drop('volume', axis=1)
print(twomin)

In [ ]:
backtest = twomin.copy()
backtest.insert(len(backtest.columns), 'buy', '')
backtest.insert(len(backtest.columns), 'sell', '')
print(backtest)

In [ ]:
buypos = False
sellpos = False
for ind, rows in backtest.iterrows():
    if ind == 0:
        continue
    if buypos == False and sellpos == False:
        if backtest.loc[ind, 'high'] >= backtest.loc[ind-1, 'high']:
            backtest.loc[ind, 'buy'] = 'OPEN'
            posopen = buyopen = backtest.loc[ind-1, 'high']
            buytarget = buyopen * 1.01
            buystoploss = buyopen * 0.99
            entrytime = backtest.loc[ind, 'time']
            buypos = True
            position = 'long'
            if backtest.loc[ind, 'close'] - backtest.loc[ind, 'open'] >=0:
                buycandle = 'G'
        elif backtest.loc[ind, 'low'] <= backtest.loc[ind-1, 'low']:
            backtest.loc[ind, 'sell'] = 'OPEN'
            posopen = sellopen = backtest.loc[ind-1, 'low']
            selltarget = sellopen * 0.99
            sellstoploss = sellopen * 1.01
            entrytime = backtest.loc[ind, 'time']
            sellpos = True
            position = 'short'
            if backtest.loc[ind, 'close'] - backtest.loc[ind, 'open'] <=0:
                sellcandle = 'R'
    if buypos == True:
        if backtest.loc[ind, 'high'] >= buytarget:
            backtest.loc[ind, 'buy'] = backtest.loc[ind, 'buy'] + ' +CLOSE'
            exittime = backtest.loc[ind, 'time']
            exitprice = buytarget
            success = 'YES'
            break
        elif backtest.loc[ind, 'low'] <= buystoploss and not buycandle == 'G':
            backtest.loc[ind, 'buy'] = backtest.loc[ind, 'buy'] + ' -CLOSE'
            exittime = backtest.loc[ind, 'time']
            exitprice = buystoploss
            success = 'NO'
            break
    if sellpos == True:
        if backtest.loc[ind, 'high'] >= sellstoploss and not sellcandle == 'R':
            backtest.loc[ind, 'sell'] = backtest.loc[ind, 'sell'] + ' -CLOSE'
            exittime = backtest.loc[ind, 'time']
            exitprice = sellstoploss
            success = 'NO'
            break
        elif backtest.loc[ind, 'low'] <= selltarget:
            backtest.loc[ind, 'sell'] = backtest.loc[ind, 'sell'] + ' +CLOSE'
            exittime = backtest.loc[ind, 'time']
            exitprice = selltarget
            success = 'YES'
            break
    buycandle = 'X'
    sellcandle = 'X'
        
print(backtest)